In [16]:
import ibm_db
import pandas
import ibm_db_dbi

In [17]:
#Replace the placeholder values with your actual Db2 hostname, username, and password:
dsn_hostname = "fbd88901-ebdb-4a4f-a32e-9822b9fb237b.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud" # e.g.: "54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud"
dsn_uid = "xxr97199"        # e.g. "abc12345"
dsn_pwd = "cSi8tkLJTBo9qfCt"      # e.g. "7dBZ3wWt9XN6$o0J"

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "bludb"            # e.g. "BLUDB"
dsn_port = "32731"                # e.g. "32733" 
dsn_protocol = "TCPIP"            # i.e. "TCPIP"
dsn_security = "SSL"              #i.e. "SSL"

In [18]:
#Create database connection
#DO NOT MODIFY THIS CELL. Just RUN it with Shift + Enter
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};"
    "SECURITY={7};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd,dsn_security)

try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )

Connected to database:  bludb as user:  xxr97199 on host:  fbd88901-ebdb-4a4f-a32e-9822b9fb237b.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud


In [19]:
#connection for pandas
pconn = ibm_db_dbi.Connection(conn)

In [20]:
#Construct the query that retrieves all rows from the CHICAGO_CRIME_DATA table
selectQuery = "select count(*) from CHICAGO_CRIME_DATA"

#retrieve the query results into a pandas dataframe
pdf = pandas.read_sql(selectQuery, pconn)

In [21]:
pdf

,1
0,533


In [22]:
#Construct the query that retrieves all rows from the CHICAGO_CRIME_DATA table
selectQuery = "select COMMUNITY_AREA_NAME from CENSUS_DATA where PER_CAPITA_INCOME < 11000"

#Execute the statement
selectStmt = ibm_db.exec_immediate(conn, selectQuery)

#Fetch the Dictionary (for the first row only) - replace ... with your code

while ibm_db.fetch_row(selectStmt) != False:
   print (" Area:",  ibm_db.result(selectStmt, "COMMUNITY_AREA_NAME"))

 Area: West Garfield Park
 Area: South Lawndale
 Area: Fuller Park
 Area: Riverdale


In [23]:
#Construct the query that retrieves all rows from the CHICAGO_CRIME_DATA table
selectQuery = "select CASE_NUMBER from CHICAGO_CRIME_DATA WHERE DESCRIPTION LIKE '%MINOR'"

#retrieve the query results into a pandas dataframe
pdf = pandas.read_sql(selectQuery, pconn)

#print just the LNAME for first row in the pandas data frame
pdf

,CASE_NUMBER
0,HL266884
1,HK238408


In [24]:
#Construct the query that retrieves all rows from the CHICAGO_CRIME_DATA table
selectQuery = "select * from CHICAGO_CRIME_DATA WHERE PRIMARY_TYPE LIKE '%KID%'"

#Execute the statement
selectStmt = ibm_db.exec_immediate(conn, selectQuery)

while ibm_db.fetch_row(selectStmt) != False:
   print ( " ID:",  ibm_db.result(selectStmt, "ID"), " case: ", ibm_db.result(selectStmt, "DESCRIPTION"))


 ID: 5276766  case:  CHILD ABDUCTION/STRANGER


In [25]:
#query statement to retrieve all rows in INSTRUCTOR table
selectQuery = "select AVG(SAFETY_SCORE) from CHICAGO_PUBLIC_SCHOOLS"

#retrieve the query results into a pandas dataframe
pdf = pandas.read_sql(selectQuery, pconn)

#print just the LNAME for first row in the pandas data frame
pdf

,1
0,49


In [26]:
#query statement to retrieve all rows in INSTRUCTOR table
selectQuery = "select COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY from CENSUS_DATA order by PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC LIMIT 5"

#retrieve the query results into a pandas dataframe
pdf = pandas.read_sql(selectQuery, pconn)

#print just the LNAME for first row in the pandas data frame
pdf

,COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
0,Riverdale,56.5
1,Fuller Park,51.2
2,Englewood,46.6
3,North Lawndale,43.1
4,East Garfield Park,42.4


In [27]:
#query statement to retrieve all rows in INSTRUCTOR table
selectQuery = "select COMMUNITY_AREA_NAME, HARDSHIP_INDEX, COMMUNITY_AREA_NUMBER from CENSUS_DATA where HARDSHIP_INDEX = (select MAX(HARDSHIP_INDEX) from CENSUS_DATA)"

#retrieve the query results into a pandas dataframe
pdf = pandas.read_sql(selectQuery, pconn)

#print just the LNAME for first row in the pandas data frame
pdf

,COMMUNITY_AREA_NAME,HARDSHIP_INDEX,COMMUNITY_AREA_NUMBER
0,Riverdale,98,54


In [28]:
#query statement to retrieve all rows in INSTRUCTOR table
selectQuery = "select COMMUNITY_AREA_NAME, HARDSHIP_INDEX from CENSUS_DATA where HARDSHIP_INDEX = (select MAX(HARDSHIP_INDEX) from CENSUS_DATA)"

#retrieve the query results into a pandas dataframe
pdf = pandas.read_sql(selectQuery, pconn)

#print just the LNAME for first row in the pandas data frame
pdf

,COMMUNITY_AREA_NAME,HARDSHIP_INDEX
0,Riverdale,98


In [29]:
# list the school names, community names and average attendance for communities with a hardship index of 98. 
selectQuery = "select P.NAME_OF_SCHOOL, P.AVERAGE_STUDENT_ATTENDANCE, P.AVERAGE_TEACHER_ATTENDANCE, D.COMMUNITY_AREA_NAME FROM CHICAGO_PUBLIC_SCHOOLS \
                    AS P LEFT JOIN CENSUS_DATA AS D ON P.COMMUNITY_AREA_NUMBER=D.COMMUNITY_AREA_NUMBER WHERE D.HARDSHIP_INDEX='98'"

#retrieve the query results into a pandas dataframe
pdf = pandas.read_sql(selectQuery, pconn)

#print just the LNAME for first row in the pandas data frame
pdf

,NAME_OF_SCHOOL,AVERAGE_STUDENT_ATTENDANCE,AVERAGE_TEACHER_ATTENDANCE,COMMUNITY_AREA_NAME
0,George Washington Carver Military Academy High...,91.60%,96.40%,Riverdale
1,George Washington Carver Primary School,90.90%,94.70%,Riverdale
2,Ira F Aldridge Elementary School,92.90%,96.30%,Riverdale
3,William E B Dubois Elementary School,93.30%,94.40%,Riverdale


In [30]:
# Write and execute a SQL query to list all crimes that took place at a school. Include case number, crime type and community name. 
selectQuery = "select C.CASE_NUMBER, C.PRIMARY_TYPE, D.COMMUNITY_AREA_NAME, C.LOCATION_DESCRIPTION from CHICAGO_CRIME_DATA as C \
                LEFT JOIN CENSUS_DATA AS D ON C.COMMUNITY_AREA_NUMBER=D.COMMUNITY_AREA_NUMBER WHERE C.LOCATION_DESCRIPTION LIKE '%SCHOOL%'"

#retrieve the query results into a pandas dataframe
pdf = pandas.read_sql(selectQuery, pconn)

#print just the LNAME for first row in the pandas data frame
pdf

,CASE_NUMBER,PRIMARY_TYPE,COMMUNITY_AREA_NAME,LOCATION_DESCRIPTION
0,HK577020,NARCOTICS,Rogers Park,"SCHOOL, PUBLIC, GROUNDS"
1,HL725506,BATTERY,Lincoln Square,"SCHOOL, PUBLIC, BUILDING"
2,HH639427,BATTERY,Austin,"SCHOOL, PUBLIC, BUILDING"
3,HS200939,CRIMINAL DAMAGE,Austin,"SCHOOL, PUBLIC, GROUNDS"
4,HT315369,ASSAULT,East Garfield Park,"SCHOOL, PUBLIC, GROUNDS"
5,HP716225,BATTERY,Douglas,"SCHOOL, PUBLIC, BUILDING"
6,HL353697,BATTERY,South Shore,"SCHOOL, PUBLIC, GROUNDS"
7,HS305355,NARCOTICS,Brighton Park,"SCHOOL, PUBLIC, BUILDING"
8,JA460432,BATTERY,Ashburn,"SCHOOL, PUBLIC, GROUNDS"
9,HR585012,CRIMINAL TRESPA,Ashburn,"SCHOOL, PUBLIC, GROUNDS"
